In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
class NeuronalNetVector(nn.Module):
        def __init__(self):
            super(NeuronalNetVector, self).__init__()
            self.lin1 = nn.Linear(8,30)
            self.lin2 = nn.Linear(30,30)
            self.lin3 = nn.Linear(30,3)
        def forward(self,x):
            x = self.lin1(x)
            x = F.relu(x)
            x = F.relu(self.lin2(x))
            x = F.relu(self.lin3(x))
            x = F.softmax(x,dim=0)
            return x

In [6]:
#Die Tore der einzelnen Mannschaften pro Spiel aus den matches in eine Liste schreiben
def inputValues(gameList, df_stats,inputList):
    for match in gameList:
        homeTeam = match.iat[0,0]
        awayTeam = match.iat[0,1]
        input_values= df_stats[df_stats['HomeTeam']==homeTeam]
        input_values= input_values[input_values['AwayTeam']==awayTeam]
        input_values= input_values.reset_index(drop=True)
        input_values = input_values[:-1]
        inputList.append(input_values.loc[:,['FTHG','FTAG']])

In [7]:
#Methode um aus einer Liste eine Tensorliste mit Floatwerten generieren
def dfToTensor(dfList, tensorList):
    for df in dfList:
        tensor = torch.FloatTensor(df.values).view(8)
        tensorList.append(tensor)

In [8]:
def dfToTensor_forOutput(dfList, tensorList, viewValue):
    for df in dfList:
        tensor = torch.FloatTensor(df).view(viewValue)
        tensorList.append(tensor)

In [9]:
#Outputvalues generieren, bei einem target als Vector
def outputValuesVector(gameList, df_stats,outputList):
    for match in gameList:
        homeTeam = match.iat[0,0]
        awayTeam = match.iat[0,1]
        output_values= df_stats[df_stats['HomeTeam']==homeTeam]
        output_values= output_values[output_values['AwayTeam']==awayTeam]
        output_values= output_values.reset_index(drop=True)
        output_values = output_values.tail(1)
        output_values = output_values.loc[:,['FTHG','FTAG']]
        homegoals = output_values.loc[:,['FTHG']].iloc[0]['FTHG']
        awaygoals = output_values.loc[:,['FTAG']].iloc[0]['FTAG']
        if homegoals > awaygoals:
            df_out=[1.,0.,0.]
        if homegoals == awaygoals:
            df_out=[0.,1.,0.]
        if homegoals < awaygoals:
            df_out=[0.,0.,1.]
        outputList.append(df_out)

In [10]:
#Pfade Fabian
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/09_10-13_14.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/10_11-14_15.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/11_12-15_16.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/12_13-16_17.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/13_14-17_18.csv
#/Users/fabian-maltemoller/Developer/Ki/KI_Daten/14_15-18_19.csv

#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\09_10-13_14.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\10_11-14_15.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\11_12-15_16.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\12_13-16_17.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\13_14-17_18.csv
#C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\14_15-18_19.csv

In [11]:
def new_inputs_Vector(path,TestInputListe,TestOutputListe):
    topTeams = pd.read_csv(path, sep=';')
    df = pd.DataFrame(topTeams, columns = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR'])
    df_names = df.drop_duplicates(['HomeTeam','AwayTeam'],keep = 'last')
    df_names = df_names[['HomeTeam','AwayTeam']]
    gameListe = []
    for row in df_names.itertuples():
        data = [[row.HomeTeam,row.AwayTeam]]
        gameListe.append(pd.DataFrame(data))
    inputValues(gameListe, df,TestInputListe)
    outputValuesVector(gameListe,df,TestOutputListe)

In [12]:
inputList_0910_1314_Vector = []
outputList_0910_1314_Vector = []
new_inputs_Vector('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/09_10-13_14.csv',inputList_0910_1314_Vector,outputList_0910_1314_Vector)
#new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\09_10-13_14.csv',inputList_0910_1314,outputList_0910_1314)

inputList_1011_1415_Vector = []
outputList_1011_1415_Vector = []
new_inputs_Vector('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/10_11-14_15.csv',inputList_1011_1415_Vector,outputList_1011_1415_Vector)
#new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\10_11-14_15.csv',inputList_1011_1415,outputList_1011_1415)

inputList_1112_1516_Vector = []
outputList_1112_1516_Vector = []
new_inputs_Vector('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/11_12-15_16.csv',inputList_1112_1516_Vector,outputList_1112_1516_Vector)
#new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\11_12-15_16.csv',inputList_1112_1516,outputList_1112_1516)

inputList_1213_1617_Vector = []
outputList_1213_1617_Vector = []
new_inputs_Vector('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/12_13-16_17.csv',inputList_1213_1617_Vector,outputList_1213_1617_Vector)
#new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\12_13-16_17.csv',inputList_1213_1617,outputList_1213_1617)

inputList_1314_1718_Vector = []
outputList_1314_1718_Vector = []
new_inputs_Vector('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/13_14-17_18.csv',inputList_1314_1718_Vector,outputList_1314_1718_Vector)
#new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\13_14-17_18.csv',inputList_1314_1718,outputList_1314_1718)

inputList_1415_1819_Vector = []
outputList_1415_1819_Vector = []
new_inputs_Vector('/Users/fabian-maltemoller/Developer/Ki/KI_Daten/14_15-18_19.csv',inputList_1415_1819_Vector,outputList_1415_1819_Vector)
#new_inputs('C:\\Users\\dusti\\iCloudDrive\\Studium WI\\WS2019\\KI\\csvData\\14_15-18_19.csv',inputList_1415_1819,outputList_1415_1819)

In [13]:
#Leere Dataframes löschen
for i in range(len(inputList_1415_1819_Vector)):
    if inputList_1415_1819_Vector[i].empty:
        print(i)

11
14
32
46
54
63
85
92
97
109
116


In [14]:
inputList_1415_1819_Vector.pop(11)
inputList_1415_1819_Vector.pop(13)
inputList_1415_1819_Vector.pop(30)
inputList_1415_1819_Vector.pop(43)
inputList_1415_1819_Vector.pop(50)
inputList_1415_1819_Vector.pop(58)
inputList_1415_1819_Vector.pop(79)
inputList_1415_1819_Vector.pop(85)
inputList_1415_1819_Vector.pop(89)
inputList_1415_1819_Vector.pop(100)
inputList_1415_1819_Vector.pop(106)

outputList_1415_1819_Vector.pop(11)
outputList_1415_1819_Vector.pop(13)
outputList_1415_1819_Vector.pop(30)
outputList_1415_1819_Vector.pop(43)
outputList_1415_1819_Vector.pop(50)
outputList_1415_1819_Vector.pop(58)
outputList_1415_1819_Vector.pop(79)
outputList_1415_1819_Vector.pop(85)
outputList_1415_1819_Vector.pop(89)
outputList_1415_1819_Vector.pop(100)
outputList_1415_1819_Vector.pop(106)

print(len(inputList_1415_1819_Vector))
print(len(outputList_1415_1819_Vector))

110
110


In [16]:
train_tensor_input_vector = []
dfToTensor(inputList_0910_1314_Vector,train_tensor_input_vector)
dfToTensor(inputList_1011_1415_Vector,train_tensor_input_vector)
dfToTensor(inputList_1112_1516_Vector,train_tensor_input_vector)
dfToTensor(inputList_1213_1617_Vector,train_tensor_input_vector)
dfToTensor(inputList_1314_1718_Vector,train_tensor_input_vector)

test_tensor_input_vector= []
dfToTensor(inputList_1415_1819_Vector,test_tensor_input_vector)

In [17]:
train_tensor_vector_target = []
dfToTensor_forOutput(outputList_0910_1314_Vector,train_tensor_vector_target,3)
dfToTensor_forOutput(outputList_1011_1415_Vector,train_tensor_vector_target,3)
dfToTensor_forOutput(outputList_1112_1516_Vector,train_tensor_vector_target,3)
dfToTensor_forOutput(outputList_1213_1617_Vector,train_tensor_vector_target,3)
dfToTensor_forOutput(outputList_1314_1718_Vector,train_tensor_vector_target,3)

test_tensor_vector_target = []
dfToTensor_forOutput(outputList_1415_1819_Vector,test_tensor_vector_target,3)

In [18]:
print(len(train_tensor_input_vector))
print(len(train_tensor_vector_target))
print(len(test_tensor_input_vector))
print(len(test_tensor_vector_target))

710
710
110
110


In [19]:
net = NeuronalNetVector()

In [20]:
criterion = nn.L1Loss()

In [21]:
optimizer = torch.optim.SGD(net.parameters(),lr =0.01, momentum = 0.9)

In [22]:
for epoch in range(50):
    running_loss = 0.0
    for i in range(len(train_tensor_input_vector)):
        optimizer.zero_grad()
        outputs = net(train_tensor_input_vector[i])
        loss = criterion(outputs, train_tensor_vector_target[i])
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 710 == 709:
            print (epoch + 1 , i+1 , running_loss / 710)
            running_loss = 0.0

1 710 0.37375617052623117
2 710 0.35997360629958575
3 710 0.3598085324565285
4 710 0.3597457067240242
5 710 0.3597110166187614
6 710 0.35968751556821826
7 710 0.35967012005917814
8 710 0.3596562276913858
9 710 0.35964511328790955
10 710 0.3596349144922659
11 710 0.3596255880835351
12 710 0.3596172451952893
13 710 0.35960890564330505
14 710 0.3596004774402667
15 710 0.3595920954676129
16 710 0.35958349144071894
17 710 0.35957296801629285
18 710 0.3595618261946276
19 710 0.3595489097220784
20 710 0.35953352939695526
21 710 0.3595143094073945
22 710 0.3594893062111187
23 710 0.35945110073151243
24 710 0.35939449934515894
25 710 0.35929463311817383
26 710 0.35881483266167036
27 710 0.35836632125860207
28 710 0.35820638628535384
29 710 0.3580170861068661
30 710 0.3575229769903439
31 710 0.3573071462289506
32 710 0.357350091315056
33 710 0.35758826514731384
34 710 0.35605824807845987
35 710 0.3748105706850607
36 710 0.350516200282685
37 710 0.349531768747468
38 710 0.3415484606719097
39 710 

In [23]:
#Quantitative Evaluierung 
correct = 0
total = 0
with torch.no_grad():
    for i in range(len(test_tensor_input_vector)) :
        outputs = net(test_tensor_input_vector[i])
        if (torch.argmax(outputs) == torch.argmax(test_tensor_vector_target[i])):
            correct = correct+1
print(correct / len(test_tensor_input_vector))

0.4909090909090909
